# Content-based Filtering

The content-based filtering recommendation algorithm suggests items similar to ones the user has liked before by analyzing the features of the items and the user’s preferences. It compares the content (e.g., product descriptions, keywords, tags) of items with a user’s historical interactions (such as movies they have watched or articles they have read) and suggests items with similar attributes.

Imagine a user watching a sci-fi movie, and the system extracts features like “genre: sci-fi,” “director: Christopher Nolan,” and “themes: space exploration.” The system would then recommend other movies sharing these features, e.g., another sci-fi film with similar themes and directors.

In this notebook, we would take a very simple and straight-forward approach to implement Content-based Filtering. We make recommendations by finding the most similar items to the user's last interacted item, where similarity is defined based on the textual description of the items. The score is the similarity between the target item and the neighbors.

Specifically, we will:
- Select a few description fields from the item metadata
- Transform the features into numerical format, e.g. TFIDF for text data, Multi-Hot Encoders for categorical features
- Concat and normalize the transformation output
- Use cosine similarity to look up neighbor items and use it as the prediction score

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import dill
import mlflow
import numpy as np  # required for the scikit-learn pipeline to work
import pandas as pd
import torch
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from sklearn.preprocessing import MinMaxScaler

load_dotenv()

sys.path.insert(0, "..")


from src.eval import (
    create_label_df,
    create_rec_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.id_mapper import IDMapper
from src.model import ContentBased
from src.train_utils import map_indice

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L4 - Reco Algo"
    run_name: str = "005-content-based"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-25 02:34:24.351 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L4 - Reco Algo - run 005-content-based...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L4 - Reco Algo",
  "run_name": "005-content-based",
  "notebook_persist_dp": "/home/jupyter/frostmourne/reco-algo/notebooks/data/005-content-based",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Implement

In [4]:
def init_model(train_item_features, device):
    model = ContentBased(item_features=train_item_features, device=device)
    return model

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
# device = 'cpu'
logger.info(f"Using {device} device")

2024-09-25 02:34:24.570 | INFO     | __main__:<module>:7 - Using cuda device


In [6]:
with open("../data/item_metadata_pipeline.dill", "rb") as f:
    item_metadata_pipeline = dill.load(f)

# Test implementation

In [7]:
# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
ratings = [1, 4, 5, 3, 2]
timestamps = [0, 1, 2, 3, 4]
main_category = [
    "All Electronics",
    "Video Games",
    "All Electronics",
    "Video Games",
    "Unknown",
]
title = ["All Electronics", "Video Games", "All Electronics", "Video Games", "Unknown"]
description = [[], [], ["Video games blah blah"], [], ["blah blah"]]
categories = [[], ["Headsets"], ["Video Games"], [], ["blah blah"]]
price = ["from 14.99", "14.99", "price: 9.99", "20 dollars", "None"]

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.rating_col: ratings,
        args.timestamp_col: timestamps,
        "main_category": main_category,
        "title": title,
        "description": description,
        "categories": categories,
        "price": price,
    }
)
# Drop duplicated item features so that the ContentBased model will fit correctly in terms of index mapping
fit_df = train_df.drop_duplicates(subset=["item_indice"])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)

val_user_indices = [0, 1, 2]
val_item_indices = [2, 1, 2]
val_ratings = [2, 4, 5]
val_timestamps = [5, 6, 7]
val_main_category = ["All Electronics", "Video Games", "All Electronics"]
val_title = ["All Electronics", "Video Games", "All Electronics"]
val_description = [["Video games blah blah"], [], ["Video games blah blah"]]
val_categories = [["Video Games"], ["Headsets"], ["Video Games"]]
val_price = ["price: 9.99", "14.99", "price: 9.99"]

val_df = pd.DataFrame(
    {
        "user_indice": val_user_indices,
        "item_indice": val_item_indices,
        args.rating_col: val_ratings,
        args.timestamp_col: val_timestamps,
        "main_category": val_main_category,
        "title": val_title,
        "description": val_description,
        "categories": val_categories,
        "price": val_price,
    }
)
val_item_features = item_metadata_pipeline.transform(val_df).astype(np.float32)

In [8]:
n_users = len(set(user_indices))
n_items = len(set(item_indices))

model = init_model(train_item_features, device)

items1 = [1, 2]
items2 = [0, 3]
predictions = model.predict(items1, items2)
print(predictions)

print("\n\n")

users = [0, 1]
anchor_items = [2, 3]
recommendations = model.recommend(users, anchor_items, k=args.top_K)
print(recommendations)

tensor([0.1171, 0.0386], device='cuda:0')





Generating Recommendations:   0%|          | 0/2 [00:00<?, ?it/s]

{'user_indice': [0, 0, 0, 0, 1, 1, 1, 1], 'recommendation': [0, 4, 1, 3, 1, 4, 0, 2], 'score': [0.6484283804893494, 0.1365058422088623, 0.03903510421514511, 0.03860381618142128, 0.9609925150871277, 0.3168308436870575, 0.16951429843902588, 0.03860381618142128]}


# Prep data

In [9]:
def get_last_item(df, item_sequence_col="item_sequence"):
    return df.assign(
        last_item_indice=lambda df: df[item_sequence_col].apply(lambda s: s[-1])
    )

In [10]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm = IDMapper().load("../data/idm.json")
assert (val_df[args.timestamp_col].min() - train_df[args.timestamp_col].max()) > 0
val_timestamp = train_df[args.timestamp_col].max() + 1
print(f"{val_timestamp=}")

val_timestamp=np.int64(1628641464793)


In [11]:
train_df = train_df.pipe(get_last_item)
val_df = val_df.pipe(get_last_item)
full_df = pd.concat([train_df, val_df], axis=0)
user_ids = train_df[args.user_col].values
item_ids = train_df[args.item_col].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-09-25 02:34:33.063 | INFO     | __main__:<module>:9 - len(unique_user_ids)=20,366, len(unique_item_ids)=4,696


In [12]:
train_df = train_df.pipe(map_indice, idm, args.user_col, args.item_col)
val_df = val_df.pipe(map_indice, idm, args.user_col, args.item_col)
full_df = full_df.pipe(map_indice, idm, args.user_col, args.item_col)

In [13]:
train_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice
0,AEIE6SAJAELLZA4BLSNPOYLEWDXA,B013P0HW7K,0.0,1455882952000,13174,264,Computers,Razer Orbweaver Chroma Gaming Keypad: Mechanic...,[The Razer orb Weaver is the world's first mec...,"[Video Games, PC, Accessories, Gaming Keyboards]",None,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 688...",41.0
1,AEMZFONWV7KHCF2CPIVKKNCKHLLQ,B0086VPUHI,0.0,1416423128000,7749,3275,Video Games,Grand Theft Auto V: Premium Edition - Xbox One...,[The Grand Theft Auto V: Premium Edition inclu...,"[Video Games, Xbox One, Games]",15.0,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....",4078.0
2,AFH25PNHGY6BUBZBK4OAL4QJBYGQ,B00YJJ0OQS,0.0,1468541348000,9440,2318,Video Games,Xbox One Wireless Controller [Without Bluetooth],"[Experience the unique intensity, precision an...","[Video Games, Xbox One, Accessories, Controlle...",71.77,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....",1435.0
3,AGUN3QQ3WECXH6D3FYKKFI5HRNNA,B00BN5T30E,0.0,1509679302842,12165,4438,Video Games,Assassin's Creed IV Black Flag - PC,"[From the Manufacturer, Assassin's Creed, ®, I...","[Video Games, PC, Games]",43.89,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....",-1.0
4,AFAUFOADANHRMH3GR2FA73A6NHIA,B08VFQ3XJX,5.0,1408655680000,18519,3311,Video Games,Final Fantasy X,"[Product Description, Final Fantasy X finally ...","[Video Games, Legacy Systems, PlayStation Syst...",20.0,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 2385.0, 1...",1834.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
340085,AGYL3DDXAR56G6XEAQVCSQCGZSXA,B001ELJFGE,0.0,1033148855000,9450,2212,Video Games,Army of Two - Playstation 3,"[Product description, Delivering a groundbreak...","[Video Games, Legacy Systems, PlayStation Syst...",35.9,"[920.0, 739.0, 677.0, 4093.0, 3808.0, 1745.0, ...",1201.0
340086,AEQDXMB264KG4OZUZL7NAJFZGADA,B001TOQ8NO,0.0,1466618460000,5452,3466,Video Games,Wii MotionPlus,[],"[Video Games, Legacy Systems, Nintendo Systems...",46.98,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 3138.0, 3...",2539.0
340087,AEYD3JIWG72NBXR4BXSEZX3ZMA6Q,B06XHMPPLH,0.0,1417535860000,18134,4019,Video Games,NieR: Automata - Playstation 4,[],"[Video Games, PlayStation 4, Games]",25.93,"[506.0, 3116.0, 673.0, 1272.0, 4315.0, 1973.0,...",3995.0
340088,AEFJKBRX2TPPMMJ53DKHANKRYDXQ,B08N6NMGNB,5.0,1510350967758,6010,928,Video Games,Thrustmaster T300 RS - Gran Turismo Edition Ra...,[Works with PS5 games (PS5 games compatibility...,"[Video Games, Legacy Systems, PlayStation Syst...",449.0,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 237...",301.0


# Fit

In [14]:
fit_df = train_df.drop_duplicates(subset=[args.item_col])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)
model = init_model(train_item_features, device)

# Predict

In [15]:
user_id = val_df.sample(1)[args.user_col].values[0]
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
test_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice
92,AEYM2QL3ZZ6K6PDWEEVNP5CHH4AA,B0000690ZP,0.0,1642500098052,3454,1735,Video Games,Madden NFL 2003 - Playstation 2,"[Product Description, EA heralds the return of...","[Video Games, Legacy Systems, PlayStation Syst...",14.0,"[-1, -1, -1, 2159, 1777, 989, 767, 4692, 3790,...",1744
900,AEYM2QL3ZZ6K6PDWEEVNP5CHH4AA,B08F4D36D9,5.0,1642500098052,3454,4246,Video Games,Pokémon Sword + Pokémon Sword Expansion Pass -...,[Begin your adventure in the Galar region and ...,"[Video Games, Nintendo Switch, Games]",129.99,"[-1, -1, -1, 2159, 1777, 989, 767, 4692, 3790,...",1744
1010,AEYM2QL3ZZ6K6PDWEEVNP5CHH4AA,B072MR3V1J,0.0,1641003910692,3454,65,Video Games,HORI Nintendo Switch Super Mario Analog Caps O...,[],[],None,"[-1, -1, -1, -1, 2159, 1777, 989, 767, 4692, 3...",3790
1520,AEYM2QL3ZZ6K6PDWEEVNP5CHH4AA,B09CG15F86,3.0,1641003910692,3454,1744,Computers,Razer Doubleshot PBT Keycap Upgrade Set for Me...,[Enjoy durability without cramping your style....,"[Video Games, PC, Accessories, Gaming Keyboards]",None,"[-1, -1, -1, -1, 2159, 1777, 989, 767, 4692, 3...",3790


In [16]:
item_id = test_df.loc[lambda df: df[args.rating_col].gt(0)][args.item_col].values[0]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
anchor_item_indice = test_df["last_item_indice"].values[0]
item_indice = idm.get_item_index(item_id)
model.predict([item_indice], [anchor_item_indice])

2024-09-25 02:34:38.582 | INFO     | __main__:<module>:2 - Test predicting before training with user_id = AEYM2QL3ZZ6K6PDWEEVNP5CHH4AA and parent_asin = B08F4D36D9


tensor([-0.0114], device='cuda:0')

# Recommend

In [17]:
val_anchor_item_indices = val_df["last_item_indice"].values
val_user_indices = val_df["user_indice"].values

In [18]:
recommendations = model.recommend(val_user_indices, val_anchor_item_indices, args.top_K)

Generating Recommendations:   0%|          | 0/1898 [00:00<?, ?it/s]

# Evaluate

## Ranking metrics

In [19]:
recommendations_df = pd.DataFrame(recommendations).pipe(
    create_rec_df, idm, args.user_col, args.item_col
)
recommendations_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,5473,1299,0.423344,75.0,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B0017KIBAI
1,5473,718,0.415539,81.0,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B015TL6PGM
2,5473,4118,0.396596,87.0,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B00000K1AF
3,5473,2769,0.377835,107.0,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B002I0K74Y
4,5473,4583,0.370945,117.0,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B00BIIE3B0
...,...,...,...,...,...,...
189795,2919,3325,0.048900,192.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B00006I02Z
189796,2919,4390,0.048401,194.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B008J1372A
189797,2919,3346,0.048307,196.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B08HB9TCVG
189798,2919,158,0.047842,198.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B07NQ1B97S


In [20]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

,user_id,parent_asin,rating,rating_rank
303,AEMA3SW3WPNLEH3IACW23K2ZSUFA,B09JDLC31H,4.0,1.0
1339,AEURBISVS35ALE7YQLR5L4K7AHCA,B07QQ8N7LL,1.0,1.0
1648,AFX6IXYCJCVUAYTSPTCXHJXY3ZTQ,B076B995TL,5.0,1.0
1286,AGMBK7EWZQWYI74JQMATQGZZRFUQ,B0BL9CD53Z,4.0,1.0
1727,AEOY2365QPPEVDTOXL6N7ZA4NSAA,B00PDRZG9U,5.0,1.0
...,...,...,...,...
301,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B006QRNKOO,0.0,18.0
1832,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B017C6OK7S,0.0,18.0
1875,AFB6FYPPCN33UMUU5536IHXNOHCQ,B00K0NV5J2,0.0,18.0
1501,AFB6FYPPCN33UMUU5536IHXNOHCQ,B00308JV6I,0.0,19.0


In [21]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
60,3199.0,316.0,0.973644,1,AE2AZ2MNROPF33U6SS53VI22OXJA,B0039LME9W,0,NaN
61,3199.0,316.0,0.973644,2,AE2AZ2MNROPF33U6SS53VI22OXJA,B0039LME9W,0,NaN
179,3199.0,174.0,0.772048,3,AE2AZ2MNROPF33U6SS53VI22OXJA,B07Q63K8QY,0,NaN
180,3199.0,174.0,0.772048,4,AE2AZ2MNROPF33U6SS53VI22OXJA,B07Q63K8QY,0,NaN
197,3199.0,2917.0,0.722684,5,AE2AZ2MNROPF33U6SS53VI22OXJA,B0BJ6XP2VW,0,NaN
...,...,...,...,...,...,...,...,...
191576,11875.0,234.0,0.086558,196,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B01I59OC92,0,NaN
191597,11875.0,4444.0,0.086356,197,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B0728L4V8P,0,NaN
191598,11875.0,4444.0,0.086356,198,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B0728L4V8P,0,NaN
191591,11875.0,3467.0,0.085818,199,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B071LQD2Y7,0,NaN


In [22]:
ranking_report = log_ranking_metrics(args, eval_df)

/home/jupyter/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:61: RuntimeWarning: invalid value encountered in divide
  return (1 + beta_sqr) * precision_arr * recall_arr / (beta_sqr * precision_arr + recall_arr)


## Classification metrics

In [23]:
val_anchor_item_indices = val_df["last_item_indice"].values
val_item_indices = val_df["item_indice"].values

In [24]:
classifications = (
    model.predict(val_item_indices, val_anchor_item_indices)
    .cpu()
    .detach()
    .numpy()
    .reshape(-1, 1)
)
classifications = MinMaxScaler(feature_range=(0, 1)).fit_transform(classifications)

In [25]:
eval_classification_df = val_df.assign(
    classification_proba=classifications,
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice,classification_proba,label
0,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B001F8188Y,0.0,1640980885681,5473,2668,Video Games,Nintendo DS Lite Cobalt / Black,"[Product Description, Nintendo DS Lite Cobalt ...","[Video Games, Legacy Systems, Nintendo Systems...",117.99,"[-1, -1, 3287, 169, 2617, 2413, 3088, 3283, 28...",2272,0.125774,0
1,AHH7QSUK7NCWVYXT4DGC3WD2QMAQ,B07GP2CV85,0.0,1646754495061,8164,2501,Video Games,Metro Exodus: Day One Edition - PlayStation 4,"[Metro Exodus is an epic, story driven first p...","[Video Games, PlayStation 4, Games]",39.99,"[3105, 2926, 4020, 1750, 880, 2559, 689, 4467,...",3552,0.113737,0
2,AGMBK7EWZQWYI74JQMATQGZZRFUQ,B0012N7BSI,0.0,1628828874493,4941,3774,Video Games,Madden NFL 09 - Playstation 3,"[Product description, Madden NFL 09 delivers i...","[Video Games, Legacy Systems, PlayStation Syst...",21.06,"[-1, -1, -1, 2094, 2112, 161, 1649, 1920, 1729...",1456,0.109740,0
3,AF2L5SZ5P34FX44NEOYKLMY6WSTA,B0080CAOYM,0.0,1629274141747,4741,2067,Video Games,Call of Duty: Black Ops 2 [Download],"[Zombies, The Biggest Zombies Experience to Da...","[Video Games, PC, Games]",None,"[-1, -1, -1, -1, 612, 2050, 2080, 3583, 785, 201]",201,0.128375,0
4,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B0BLFYF8K2,4.0,1630263342566,17628,297,Computers,"Logitech G600 MMO Gaming Mouse, RGB Backlit, 2...","[With 20 buttons, the Logitech G600 MMO Gaming...","[Video Games, PC, Accessories, Gaming Mice]",37.99,"[2186, 3468, 4604, 4223, 2586, 2262, 1549, 199...",4571,0.139212,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AFUWPAK6VCGEL2OVIL2YGZNFQJZQ,B08N6NCR3Q,4.0,1642699950266,16864,76,Video Games,Thrustmaster T 16000M SPACE SIM DUO STICK (PC),[The THRUSTMASTER T.16000M FCS Space Sim Duo c...,"[Video Games, PC, Accessories, Controllers, Fl...",119.51,"[-1, -1, -1, -1, 4239, 2038, 3617, 4672, 2949,...",175,0.155584,1
1894,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,6281,2502,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 4391, 2658, 2729, 3312, 3222, 3759, 1892,...",4457,0.114196,1
1895,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,8504,4485,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 4395, 2664, 678, 1806, 4185]",4185,0.072317,1
1896,AHUA2BO4FFD3VLGEOIF4TGJYCOBQ,B01GY35GIM,0.0,1647432520927,16980,758,Video Games,Titanfall 2 - SteelBook Edition - PlayStation 4,"[I bought it about 2 weeks ago, now I have two...","[Video Games, PlayStation 4, Games]",None,"[2375, 1501, 4639, 2856, 4680, 4401, 950, 4074...",4586,0.100002,0


In [26]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

# Clean up

In [27]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/25 02:34:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run 005-content-based at: http://localhost:5003/#/experiments/2/runs/9a3d1d4d25ec4755918313a18d360045.
2024/09/25 02:34:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/2.
